ENTRENAMIENTO Y EVALUACIÓN

In [1]:
import pandas as pd
import numpy as np
import collections as cols
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

2024-11-24 00:26:50.437106: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:26:50.502213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 00:26:50.503915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-24 00:26:51.585218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#1 Cargar los datos preprocesados
file_path = 'databases/Base de datos para desarrollo v2_dest(preprocesada).csv'
df = pd.read_csv(file_path)

In [3]:
#2 Separar las características de entrada y de salida (objetivo)
input = df.drop(columns=['__destino'])
output = df['__destino']

print(input)
print(output)

        __temperatura   __pulso     __pas     __pad   __sat02
0            0.229959 -0.141229 -0.090837 -0.514129  0.166636
1           -4.667926 -3.667914 -2.134067 -2.059868 -6.671051
2           -4.667926 -1.223280  0.405649  0.495334  0.236409
3            0.229959  1.902645  1.379525  1.094702  0.306181
4            0.203121  0.980898 -0.530036  0.179877  0.236409
...               ...       ...       ...       ...       ...
189077       0.109189  0.099227 -0.186315 -0.514129  0.306181
189078       0.068933  0.540062 -0.186315 -0.514129  0.027092
189079       0.082352 -0.702292  0.653892  0.810791  0.166636
189080       0.162865 -0.501913  0.443840  0.400697  0.236409
189081       0.229959 -1.062976  0.138310 -0.324855  0.306181

[189082 rows x 5 columns]
0         0
1         0
2         0
3         0
4         0
         ..
189077    1
189078    1
189079    1
189080    1
189081    1
Name: __destino, Length: 189082, dtype: int64


In [4]:
# Convertir la salida a categorías
output = to_categorical(output)
print(output)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [5]:
#3 Dividir los datos en conjuntos de entrenamiento 80% y prueba 20% 
input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.2, random_state=42)

In [6]:
#4 Construir el modelo dela red neuronal (Perceptron multicapa)
def MLP_NN():
    NumNeurons = 7
    model = Sequential()
    model.add(Dense(64, input_dim=input_train.shape[1]))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(output.shape[1], activation='softmax'))  # Usar 'softmax' para clasificación multiclase

    #opt =  keras.optimizers.Adam(learning_rate=0.001)

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [7]:
#5 Entrenar el modelo
n_epochs = 500
#network = MLP_NN()
#train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2)


network = MLP_NN()
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
train = network.fit(input_train, output_train, epochs=n_epochs, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


2024-11-24 00:26:53.180655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-24 00:26:53.181951: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/500
3782/3782 [==============================] - 6s 1ms/step - loss: 0.6540 - accuracy: 0.6076 - val_loss: 0.6273 - val_accuracy: 0.6373
Epoch 2/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6339 - accuracy: 0.6319 - val_loss: 0.6226 - val_accuracy: 0.6365
Epoch 3/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6304 - accuracy: 0.6340 - val_loss: 0.6203 - val_accuracy: 0.6414
Epoch 4/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6283 - accuracy: 0.6372 - val_loss: 0.6192 - val_accuracy: 0.6410
Epoch 5/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6272 - accuracy: 0.6389 - val_loss: 0.6182 - val_accuracy: 0.6450
Epoch 6/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6261 - accuracy: 0.6383 - val_loss: 0.6171 - val_accuracy: 0.6444
Epoch 7/500
3782/3782 [==============================] - 5s 1ms/step - loss: 0.6255 - accuracy: 0.6395 - val_loss: 0.6166 - val_ac

In [8]:
#6 Evaluar el modelo
loss, accuracy = network.evaluate(input_test, output_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

1182/1182 [==============================] - 1s 1ms/step - loss: 0.6117 - accuracy: 0.6492
Precisión en el conjunto de prueba: 0.65


In [9]:
#7 Predicciones
output_pred = network.predict(input_test)
print(output_pred)
output_pred_classes = np.argmax(output_pred, axis=1)
print(output_pred_classes)
output_test_classes = np.argmax(output_test, axis=1)
print(output_test_classes)


1182/1182 [==============================] - 1s 854us/step
[[0.5465424  0.45345765]
 [0.5532324  0.44676748]
 [0.48811096 0.5118891 ]
 ...
 [0.3704701  0.6295299 ]
 [0.19076915 0.80923086]
 [0.48596475 0.5140352 ]]
[0 0 1 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [10]:
#8 Generar el reporte de clasificación
print(classification_report(output_test_classes, output_pred_classes))

              precision    recall  f1-score   support

           0       0.69      0.55      0.61     18942
           1       0.62      0.75      0.68     18875

    accuracy                           0.65     37817
   macro avg       0.65      0.65      0.65     37817
weighted avg       0.66      0.65      0.65     37817



In [11]:
#9 Matriz de confusión
print(confusion_matrix(output_test_classes, output_pred_classes))

[[10487  8455]
 [ 4812 14063]]


In [12]:
# Matriz de confusión con datos de entrenamiento
output_train_pred = network.predict(input_train)
output_train_pred_classes = np.argmax(output_train_pred, axis=1)
output_train_classes = np.argmax(output_train, axis=1)
print(confusion_matrix(output_train_classes, output_train_pred_classes))

4728/4728 [==============================] - 5s 970us/step
[[41980 33619]
 [19341 56325]]
